<a href="https://colab.research.google.com/github/davideandres95/ml_comm/blob/main/tut05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Tutorial 5: Equalization in Random Noise
November 18, 2021

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn, optim

### Problem 5.2 - NN Equalizers for attenuated BPSK in AWGN
In this problem, you
train various equalizers for channel (5.4) and you compare the theoretical findings of
Section 5.1 to the input-output characteristics of trained NN equalizers.

##### 1. Linear MSE equalizer: 
implement a linear NN equalizer $f$ consisting of a single
linear neuron w.r.t. to the and train it w.r.t. the MSE loss $|X-f(\tilde{Y})|$. 
Plot the
I/O function $f$ of the trained NN and compare it to (5.21).

In [ ]:
def mapper(bits, alphabet):
    return alphabet[bits]

def awgn_channel(x, snr, seed=None):
    rng = np.random.Generator(np.random.PCG64(seed))
    power_x = np.mean(np.abs(x) ** 2)
    noise_power = power_x / snr
    noise = np.sqrt(noise_power) * rng.normal(size=x.shape) 
    return x + noise

In [ ]:
# Channel model
M = 2 # cardinality of the alphabet
b = np.random.choice(M, 10000) # bits
x = mapper(b, np.array([ -1., 1.])) # symbols
gamma = 0.7
SNRdB = 0
snr = 10**(SNRdB/10)

In [ ]:
class LinearEq(nn.Module):
    def __init__(self): 
        super().__init__()
        self.out = nn.Linear(1,1)

    def forward(self, y):
        y = self.out(y)
        return y

In [ ]:
# Initialize Network
eq = LinearEq()
# Define loss function and optimizer
loss_fn_mse = nn.MSELoss()
optimizer = optim.Adam(eq.parameters(), lr=1e-2)

In [ ]:
y = gamma* awgn_channel(x,snr)
y_t = torch.tensor(y.reshape(-1,1)).float()
x_t = torch.tensor(x.reshape(1,-1)).float()
# Trainings loop
for j in range(1000):
    x_hat = eq(y_t).reshape(1, -1)
    loss = loss_fn_mse(x_hat, x_t)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Printout and visualization
    if j % 50 == 0:
        print(f'epoch {j}: Loss = {loss.detach().numpy() :.4f}')



In [ ]:
# Plot
yy = np.arange(-2.1,2.1,0.1)
f_y = eq(torch.tensor(yy.reshape(-1,1)).float()).detach().numpy().reshape(-1,)
plt.plot(yy, f_y, label = 'NN output')
plt.plot(yy, 1/gamma/(1+1/snr)*yy, ':', label='expected')
plt.plot(yy, yy/gamma, ':', label='optimal')
plt.legend()
plt.xlabel('y')
plt.ylabel('f(y)')
plt.grid()

##### 2. Non-linear MSE equalizer
Implement a non-linear NN equalizer f consisting of several hidden layers with non-linear activations and train it w.r.t. the MSE loss $|X-f(\tilde{Y})|$. Plot the I/O function $f$ of the trained NN and compare it to (5.24).

In [ ]:
class NonLinearEq(nn.Module):
    def __init__(self, n): 
        super().__init__()
        self.h1 = nn.Linear(1,n)
        self.act1 = nn.ReLU()
        self.h2 = nn.Linear(n,n)
        self.act2 = nn.ReLU()
        self.h3 = nn.Linear(n,n)
        self.act3 = nn.ReLU()
        self.out = nn.Linear(n,1)

    def forward(self, y):
        y = self.act1(self.h1(y))
        y = self.act2(self.h2(y))
        y = self.act3(self.h3(y))
        return self.out(y)

In [ ]:
# Initialize Network
eq_nl = NonLinearEq(10)
# Define loss function and optimizer
loss_fn_mse = nn.MSELoss()
optimizer = optim.Adam(eq_nl.parameters(), lr=1e-2)

In [ ]:
y = gamma* awgn_channel(x,snr)
y_t = torch.tensor(y.reshape(-1,1)).float()
x_t = torch.tensor(x.reshape(1,-1)).float()
# Trainings loop
for j in range(1000):
    x_hat = eq_nl(y_t).reshape(1, -1)
    loss = loss_fn_mse(x_hat, x_t)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Printout and visualization
    if j % 50 == 0:
        print(f'epoch {j}: Loss = {loss.detach().numpy() :.4f}')


In [ ]:
# Plot
f_y = eq_nl(torch.tensor(yy.reshape(-1,1)).float()).detach().numpy().reshape(-1,)
plt.plot(yy, f_y, label = 'NN output')
plt.plot(yy, np.tanh(yy*snr/gamma),':', label='expected')
plt.plot(yy, yy/gamma, ':', label='optimal')
plt.xlabel('y')
plt.ylabel('f(y)')
plt.legend()
plt.grid()

##### 3. Optimal equalizer 
Use the same non-linear NN as in 2. as equalizer. Concatenate
it with a single linear neuron demapper whose parameters you fix according
to Section 5.1.1. Train the NN equalizer across the demapper proxy w.r.t. the
BCE $\text{bce}_{\log}(B,L)$. Plot the I/O function f of the trained NN and compare it to
(5.9).

In [ ]:
# Initialize Network
eq_nl = NonLinearEq(10)
# Define loss function and optimizer
loss_fn_bce = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(eq_nl.parameters(), lr=0.001)

In [ ]:
y = gamma* awgn_channel(x,snr)
sigma2 = 1/snr
y_t = torch.Tensor(y.reshape(-1,1))
b_t = torch.Tensor(b.reshape((-1,1)))
# Trainings loop
for j in range(1000):
    x_hat = eq_nl(y_t)
    loss = loss_fn_bce(2/sigma2 * x_hat, b_t)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Printout and visualization
    if j % 50 == 0:
        print(f'epoch {j}: Loss = {loss.detach().numpy() :.4f}')

In [ ]:
# Plot
f_y = eq_nl(torch.tensor(yy.reshape(-1,1)).float()).detach().numpy().reshape(-1,)

plt.plot(yy, f_y, label = 'NN output')
plt.plot(yy, yy/gamma,':', label='optimal')
plt.xlabel('y')
plt.ylabel('f(y)')
plt.legend()
plt.grid()

#### 4.
Use the NN equalizer from 1.-3. concatenated with the single linear neuron demapper
fixed according to Section 5.1.1 to verify the curves in Figure 5.4.

In [ ]:
SNRdBs = np.arange(0,10)
SNRs = 10**(SNRdBs/10)
eq_lin = []
eq_nonlin = []
eq_opt = []
for snr in SNRs:
    print(f'---- SNR is: {snr:.2f}')
    y = gamma* awgn_channel(x,snr)
    sigma2 = 1/snr
    y_t = torch.tensor(y.reshape(-1,1)).float()
    eq = LinearEq()
    optimizer = optim.Adam(eq.parameters(), lr=0.01)
    
    # Linear MSE
    # Trainings loop
    for j in range(1000):
        f_y_lin = eq(y_t).reshape(1, -1)
        loss = loss_fn_mse(f_y_lin, x_t)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Printout and visualization
        #if j % 50 == 0:
            #print(f'epoch {j}: Loss = {loss.detach().numpy() :.4f}')
    
    l = 2/sigma2 * f_y_lin.reshape(-1, 1)
    equivocation = loss_fn_bce(l, b_t).detach().numpy()/np.log(2)
    eq_lin.append(equivocation)
    print(f'equivocation linear {equivocation:.2f}')

    # Nonlinear MSE
    eq_nl = NonLinearEq(10)
    optimizer = optim.Adam(eq_nl.parameters(), lr=0.01)
    # Trainings loop
    for j in range(1000):
        f_y_nl = eq_nl(y_t).reshape(1, -1)
        loss = loss_fn_mse(f_y_nl, x_t)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Printout and visualization
        #if j % 50 == 0:
            #print(f'epoch {j}: Loss = {loss.detach().numpy() :.4f}')
    l = 2/sigma2 * f_y_nl.reshape(-1, 1)
    equivocation = loss_fn_bce(l, b_t).detach().numpy()/np.log(2)
    eq_nonlin.append(equivocation)
    print(f'equivocation nonlinear {equivocation:.2f}')

    
    
    # Optimal
    eq_nl = NonLinearEq(10)
    optimizer = optim.Adam(eq_nl.parameters(), lr=0.001)
    y_t = torch.Tensor(y.reshape(-1,1))
    b_t = torch.Tensor(b.reshape((-1,1)))
    # Trainings loop
    for j in range(1000):
        x_hat = eq_nl(y_t)
        loss = loss_fn_bce(2/sigma2 * x_hat, b_t)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Printout and visualization
        #if j % 50 == 0:
            #print(f'epoch {j}: Loss = {loss.detach().numpy() :.4f}')
    equivocation = loss.detach().numpy()/np.log(2)
    eq_opt.append(equivocation)
    print(f'equivocation optimal {equivocation:.2f}')
    



In [ ]:
plt.plot(SNRdBs, eq_lin, label='linear MSE')
plt.plot(SNRdBs, eq_nonlin, label='nonlinear MSE')
plt.plot(SNRdBs, eq_opt, label='optimal')
plt.xlabel("SNR [dB]")
plt.ylabel("Equivocation [bits]")
plt.legend()
plt.grid()